In [1]:
from data_dictionary import query_dataDict
from fileUtils import fileDownload
from sdmx import sdmx_struc

#### Read from data_dictionary all API sources (so far only two among all indicators populated)

In [2]:
# path to excel data dictionary in repo
data_dict_file = './data_dictionary/indicator_dictionary_TM_v1.xlsx'

In [3]:
# get indicators that are extracted by API (code and address in dataframe)
api_code_addr_df = query_dataDict.get_API_code_address(data_dict_file)
api_code_addr_df

,Code,Address
5,DM_BRTS,https://sdmx.data.unicef.org/ws/public/sdmxapi...
16,DM_POP_URBN,https://sdmx.data.unicef.org/ws/public/sdmxapi...


#### Get Unicef DataFlow TransMonEE country list

In [4]:
# UNICEF’s REST API data endpoint for TransMonEE Dataflow
url_endpoint = 'https://unicef.sdmxcloud.org/ws/public/sdmxapi/rest/data/ECARO,TRANSMONEE,1.0/'

In [5]:
# address for dataflow structure request
api_address = url_endpoint + 'all'
# parameters for dataflow structure request
api_params = {'format':'sdmx-json', 'detail':'structureOnly'}

# API request (dataflow structure)
d_flow_struc = fileDownload.api_request(api_address,api_params)

In [6]:
# all TransMonEE country codes are taken from its dataflow (requested by Eduard)
country_codes = sdmx_struc.get_all_country_codes(d_flow_struc.json())
# Join string of all countries for SDMX requests
country_call = '+'.join(country_codes.values())

#### Loop pandas dataframe `api_code_addr_df` and download raw data for all API sources therein

In [7]:
# raw data destination path
dest_path = './data_raw/'

In [8]:
# this is a loop along the `api_code_addr_df` pandas dataframe
for index, row in api_code_addr_df.iterrows():
    
    url_endpoint = row['Address']
    indicator_code = row['Code']
    
    # first get dataflow number of dimensions
    api_address = url_endpoint + 'all'
    api_params = {'format':'sdmx-json', 'detail':'structureOnly'}
    d_flow_struc = fileDownload.api_request(api_address,api_params)
    n_dim = sdmx_struc.get_sdmx_dim(d_flow_struc.json())
    
    # prepare to dowload data (assumes all dataflows have the same country codes)
    api_address = url_endpoint + country_call + '.' + indicator_code + '.'*(n_dim-2)
    api_params = {'format':'csv'}
    
    # now request indicator raw data
    indicator_raw = fileDownload.api_request(api_address,api_params)
    
    # write raw data to destination file if requests satisfactory
    if indicator_raw.status_code == 200:
        dest_file = f"{dest_path}{indicator_code}.csv"
        with open(dest_file, 'w+') as f:
            f.write(indicator_raw.content.decode('utf-8'))